In [1]:
import re
from pathlib import Path
import time
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.time import Time as AstroTime  # noqa
from astropy.time import TimeDelta
from functools import lru_cache

C:\Users\madha\AppData\Local\Temp\ipykernel_22124\2394361149.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%%time
raw_file = Path("NCL1/NCL1_INSPVAA_original.dat")
raw_file_text = raw_file.read_text() # len = 853715702


CPU times: total: 3.22 s
Wall time: 4.6 s


In [3]:
%%time
re_pattern = re.compile(r"#INSPVAA,.*?,.*?,.*?,.*?,.*?,.*?,.*?;(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?)\*+.*\n")
all_groups = re_pattern.findall(raw_file_text)

CPU times: total: 15.7 s
Wall time: 21.2 s


In [4]:
# num_rows = 100000
# all_groups = all_groups[:num_rows]

In [5]:
# print(f"{len(all_groups)}\n{all_groups[:10:]}")

In [6]:
# %%time
# re_pattern = re.compile(r";(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?)\*+.*\n")
# all_groups = re_pattern.findall(raw_file_text)
# print(len(all_groups), all_groups[:10])


In [7]:
HEADER = [
    "Message",
    "Port",
    "Sequence #",
    "% Idle Time",
    "Time Status",
    "Week",
    "Seconds",
    "Receiver Status",
    "Reserved",
    "Receiver S/W Version",
]

DATA_ENUM = (
    "Week",
    "Seconds",
    "Latitude",
    "Longitude #",
    "Height",
    "North Velocity",
    "East Velocity",
    "Up Velocity",
    "Roll",
    "Pitch",
    "Azimuth",
    "Status",
)
DATA_ENUM_DTYPES = ('int', 'float', 'float', 'float', 'float', 'float', 'float','float', 'float', 'float', 'float', 'object')

In [8]:
# list(zip(DATA_ENUM, DATA_ENUM_DTYPES))

In [9]:
%%time
np_array = np.array(all_groups, dtype=list(zip(DATA_ENUM, DATA_ENUM_DTYPES)))
# np_array

CPU times: total: 10.8 s
Wall time: 23.6 s


In [10]:
# %%time
# df = pd.DataFrame(np_array)
# df

In [11]:
# %%time
# df = pd.DataFrame(all_groups, columns=DATA_ENUM)

In [12]:
# from gnatss.utilities.time import gpsws_to_time

In [13]:
# %%time
# df['timestamp1'] = list(map(gpsws_to_time, df["Week"], df["Seconds"]))

In [14]:
# %%time
# df['timestamp2'] = df.apply(lambda x: gpsws_to_time(week = x["Week"], seconds=x["Seconds"]), axis=1)

In [15]:
# a = u.wk
# a = (u.wk).to(u.d)
# print(f"{a=} {type(a)=}")
# a = 10*a
# print(f"{a=} {type(a)=}")

In [16]:
# %%time
# def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
#     # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
#     week_shape = week.shape
#     gps_epoch = AstroTime(0, format="gps", scale="tai")

#     astropy_days_in_week = (u.wk).to(u.d)

    
#     num_weeks = week * np.full(week.shape, astropy_days_in_week)


#     def calc_week_timedelta(x):
#         return TimeDelta(x, format="jd", scale="tai")

#     week_time = np.vectorize(calc_week_timedelta, cache=True)(num_weeks) + np.full(week.shape, gps_epoch)


#     def calc_week_start(x):
#         return AstroTime(x.strftime("%Y-%m-%d"), format="iso", scale="tt")
        
#     week_start = np.vectorize(calc_week_start, cache=True)(week_time)


#     def calc_sec_timedelta(x):
#         return TimeDelta(x, format="sec")

#     sec_timedelta = np.vectorize(calc_sec_timedelta, cache=True)(seconds)


#     final_time = week_start + sec_timedelta


#     return final_time


#     # return final_time

# df['timestamp3'] = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])

In [17]:
# %%time
# def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
#     # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
#     week_shape = week.shape
#     gps_epoch = AstroTime(0, format="gps", scale="tai")

#     astropy_days_in_week = (u.wk).to(u.d)

    
#     num_weeks = week * np.full(week.shape, astropy_days_in_week)


#     @lru_cache(maxsize=8096)
#     def calc_week_timedelta(x):
#         return TimeDelta(x, format="jd", scale="tai")

#     week_time = np.vectorize(calc_week_timedelta)(num_weeks) + np.full(week.shape, gps_epoch)


#     @lru_cache(maxsize=8096)
#     def calc_week_start(x):
#         return AstroTime(x.strftime("%Y-%m-%d"), format="iso", scale="tt")
        
#     week_start = np.vectorize(calc_week_start)(week_time)


#     @lru_cache(maxsize=8096)
#     def calc_sec_timedelta(x):
#         return TimeDelta(x, format="sec")

#     sec_timedelta = np.vectorize(calc_sec_timedelta)(seconds)


#     final_time = week_start + sec_timedelta


#     return final_time


#     # return final_time

# df['timestamp4'] = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])

In [18]:
# %%time
# week = np_array["Week"]
# seconds = np_array["Seconds"]
# unique_week, unique_index, index_vals = np.unique(week, return_inverse=True, return_index=True)
# unique_week

In [19]:
# %%time
# gps_epoch = AstroTime(0, format="gps", scale="tai")
# num_days = (week * u.wk).to(u.d)

# week_time = gps_epoch + TimeDelta(num_days, format="jd", scale="tai")
# week_time

In [20]:
# %%time
# week_start = AstroTime(week_time.strftime("%Y-%m-%d"), format="iso", scale="tt")
# week_start

In [21]:
# %%time
# final_time = week_start + TimeDelta(seconds, format="sec")
# final_time

In [22]:
# np.array_equal(final_time, df['timestamp1'].values)

In [23]:
# %%time
# def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
#     # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
#     gps_epoch = AstroTime(0, format="gps", scale="tai")
#     # Convert week to days
#     num_days = (week * u.wk).to(u.d)
#     # Add days to time 0, includes leap seconds
#     week_time = gps_epoch + TimeDelta(num_days, format="jd", scale="tai")
#     # Get the week start time exactly at midnight, doesn't include leap seconds, TT scale
#     week_start = AstroTime(week_time.strftime("%Y-%m-%d"), format="iso", scale="tt")
#     # Add seconds to beginning of week
#     final_time = week_start + TimeDelta(seconds, format="sec")
#     return final_time

# timestamps = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])

In [24]:
# %%time

# def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
#     # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
#     gps_epoch = AstroTime(0, format="gps", scale="tai")
#     # Convert week to days
#     num_days = (week * u.wk).to(u.d)
#     # Add days to time 0, includes leap seconds
#     week_time = gps_epoch + TimeDelta(num_days, format="jd", scale="tai")
#     # Get the week start time exactly at midnight, doesn't include leap seconds, TT scale
#     week_start = AstroTime(week_time.strftime("%Y-%m-%d"), format="iso", scale="tt")
#     # Add seconds to beginning of week
#     final_time = week_start + TimeDelta(seconds, format="sec")
#     return final_time

# timestamps = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])
# timestamps

In [25]:
from gnatss.utilities.time import AstroTime

In [26]:
%%time

def week_to_timestamp(week: np.ndarray, seconds: np.ndarray):
    # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
    unique, unique_index = np.unique(week, return_inverse=True)
    gps_epoch = AstroTime(0, format="gps", scale="tai")
    # Convert week to days
    num_days = (unique * u.wk).to(u.d)
    # Add days to time 0, includes leap seconds
    week_time = gps_epoch + TimeDelta(num_days, format="jd", scale="tai")
    # Get the week start time exactly at midnight, doesn't include leap seconds, TT scale
    week_start = AstroTime(week_time.strftime("%Y-%m-%d"), format="iso", scale="tt")
    # Add seconds to beginning of week
    final_time = week_start[unique_index]  + TimeDelta(seconds, format="sec")
    final_time_j2000 = final_time.unix_j2000
    print(f"{type(final_time_j2000[1])=}")
    
    return final_time_j2000

final_time_vals = week_to_timestamp(np_array["Week"], np_array["Seconds"])
final_time_vals.shape

type(final_time_j2000[1])=<class 'numpy.float64'>
CPU times: total: 2.23 s
Wall time: 2.95 s


(3866216,)

In [27]:
final_time_vals

array([7.12293533e+08, 7.12293533e+08, 7.12293533e+08, ...,
       7.12361848e+08, 7.12361848e+08, 7.12361848e+08])

In [28]:
final_time_vals.shape

(3866216,)

In [29]:
np.unique(final_time_vals).shape

(3866216,)

In [30]:
%%time
df = pd.DataFrame(np_array)

CPU times: total: 1.86 s
Wall time: 2.88 s


In [31]:
df["Timestamp"] = pd.Series(final_time_vals)
df

,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
0,2220,400732.75,45.302281,-124.965970,-28.3449,-0.6858,0.1014,0.2413,-3.550174,1.675695,225.485959,INS_SOLUTION_GOOD,7.122935e+08
1,2220,400732.80,45.302281,-124.965970,-28.3340,-0.6380,0.0788,0.2734,-3.540661,1.372733,225.573499,INS_SOLUTION_GOOD,7.122935e+08
2,2220,400732.85,45.302281,-124.965970,-28.3225,-0.5833,0.0534,0.2926,-3.183020,1.125801,225.649088,INS_SOLUTION_GOOD,7.122935e+08
3,2220,400732.90,45.302280,-124.965970,-28.3104,-0.5440,0.0405,0.2950,-2.585131,0.941245,225.704566,INS_SOLUTION_GOOD,7.122935e+08
4,2220,400732.95,45.302280,-124.965970,-28.2990,-0.5122,0.0346,0.2801,-1.819055,0.826606,225.732880,INS_SOLUTION_GOOD,7.122935e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3866211,2220,469048.15,45.302056,-124.965841,-23.4901,-0.9726,1.0631,-0.1158,2.405454,-1.053228,127.181676,INS_SOLUTION_GOOD,7.123618e+08
3866212,2220,469048.20,45.302055,-124.965840,-23.4962,-0.9804,1.0885,-0.1416,2.848932,-0.735741,126.521363,INS_SOLUTION_GOOD,7.123618e+08
3866213,2220,469048.25,45.302055,-124.965839,-23.5037,-0.9709,1.1255,-0.1728,3.109084,-0.440885,125.873059,INS_SOLUTION_GOOD,7.123618e+08
3866214,2220,469048.30,45.302054,-124.965838,-23.5129,-0.9542,1.1578,-0.2087,3.184167,-0.161808,125.247869,INS_SOLUTION_GOOD,7.123618e+08


In [32]:
%%time
# "‘quicksort’, ‘mergesort’, ‘heapsort’, ‘stable’"
df.sort_values(by=["Timestamp"], kind="quicksort")

CPU times: total: 625 ms
Wall time: 707 ms


,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
117138,0,1.40,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117139,0,1.45,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117140,0,1.50,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117141,0,1.55,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117142,0,1.60,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752724,2220,507677.50,45.302493,-124.965482,-25.6282,0.0802,-1.3296,0.3085,-0.950885,-1.733266,307.803709,INS_SOLUTION_GOOD,7.124005e+08
3752725,2220,507677.55,45.302493,-124.965483,-25.6152,0.0458,-1.3541,0.2389,-1.503383,-2.383026,307.580482,INS_SOLUTION_GOOD,7.124005e+08
3752726,2220,507677.60,45.302493,-124.965484,-25.6072,0.0044,-1.3617,0.1284,-2.069272,-2.864491,307.376165,INS_SOLUTION_GOOD,7.124005e+08
3752727,2220,507677.65,45.302493,-124.965485,-25.6047,-0.0325,-1.3508,0.0253,-2.584264,-3.119641,307.185557,INS_SOLUTION_GOOD,7.124005e+08


In [33]:
%%time

df.sort_values(by=["Timestamp"], kind="mergesort")

CPU times: total: 969 ms
Wall time: 1.59 s


,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
117138,0,1.40,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117139,0,1.45,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117140,0,1.50,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117141,0,1.55,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117142,0,1.60,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752724,2220,507677.50,45.302493,-124.965482,-25.6282,0.0802,-1.3296,0.3085,-0.950885,-1.733266,307.803709,INS_SOLUTION_GOOD,7.124005e+08
3752725,2220,507677.55,45.302493,-124.965483,-25.6152,0.0458,-1.3541,0.2389,-1.503383,-2.383026,307.580482,INS_SOLUTION_GOOD,7.124005e+08
3752726,2220,507677.60,45.302493,-124.965484,-25.6072,0.0044,-1.3617,0.1284,-2.069272,-2.864491,307.376165,INS_SOLUTION_GOOD,7.124005e+08
3752727,2220,507677.65,45.302493,-124.965485,-25.6047,-0.0325,-1.3508,0.0253,-2.584264,-3.119641,307.185557,INS_SOLUTION_GOOD,7.124005e+08


In [34]:
%%time

df.sort_values(by=["Timestamp"], kind="heapsort")

CPU times: total: 844 ms
Wall time: 1.43 s


,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
117138,0,1.40,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117139,0,1.45,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117140,0,1.50,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117141,0,1.55,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117142,0,1.60,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752724,2220,507677.50,45.302493,-124.965482,-25.6282,0.0802,-1.3296,0.3085,-0.950885,-1.733266,307.803709,INS_SOLUTION_GOOD,7.124005e+08
3752725,2220,507677.55,45.302493,-124.965483,-25.6152,0.0458,-1.3541,0.2389,-1.503383,-2.383026,307.580482,INS_SOLUTION_GOOD,7.124005e+08
3752726,2220,507677.60,45.302493,-124.965484,-25.6072,0.0044,-1.3617,0.1284,-2.069272,-2.864491,307.376165,INS_SOLUTION_GOOD,7.124005e+08
3752727,2220,507677.65,45.302493,-124.965485,-25.6047,-0.0325,-1.3508,0.0253,-2.584264,-3.119641,307.185557,INS_SOLUTION_GOOD,7.124005e+08


In [35]:
%%time

df.sort_values(by=["Timestamp"], kind="stable")

CPU times: total: 516 ms
Wall time: 866 ms


,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
117138,0,1.40,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117139,0,1.45,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117140,0,1.50,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117141,0,1.55,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
117142,0,1.60,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,INS_INACTIVE,-6.307632e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752724,2220,507677.50,45.302493,-124.965482,-25.6282,0.0802,-1.3296,0.3085,-0.950885,-1.733266,307.803709,INS_SOLUTION_GOOD,7.124005e+08
3752725,2220,507677.55,45.302493,-124.965483,-25.6152,0.0458,-1.3541,0.2389,-1.503383,-2.383026,307.580482,INS_SOLUTION_GOOD,7.124005e+08
3752726,2220,507677.60,45.302493,-124.965484,-25.6072,0.0044,-1.3617,0.1284,-2.069272,-2.864491,307.376165,INS_SOLUTION_GOOD,7.124005e+08
3752727,2220,507677.65,45.302493,-124.965485,-25.6047,-0.0325,-1.3508,0.0253,-2.584264,-3.119641,307.185557,INS_SOLUTION_GOOD,7.124005e+08


In [37]:
from gnatss.loaders import (
    get_atd_offsets,
    load_deletions,
    load_gps_solutions,
    load_quality_control,
    load_roll_pitch_heading,
    load_sound_speed,
    load_travel_times,
)


In [42]:
"""
files=['tests/data/2022/NCL1/208/WG_20220727/pxp_tt', 'tests/data/2022/NCL1/209/WG_20220728/pxp_tt', 'tests/data/2022/NCL1/210/WG_20220729/pxp_tt'] transponder_ids=['NCL1-1', 'NCL1-2', 'NCL1-3'] is_j2k=False time_scale='tt'
L
"""
files=['tests/data/2022/NCL1/208/WG_20220727/pxp_tt', 'tests/data/2022/NCL1/209/WG_20220728/pxp_tt', 'tests/data/2022/NCL1/210/WG_20220729/pxp_tt'] 
transponder_ids=['NCL1-1', 'NCL1-2', 'NCL1-3'] 
is_j2k=False 
time_scale='tt'

pxp_tt = load_travel_times(files, transponder_ids, is_j2k, time_scale)
pxp_tt

C:\ProgramData\Miniconda3\envs\gnatss\lib\site-packages\gnatss\loaders.py:134: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  travel_times = [pd.read_csv(i, delim_whitespace=True, header=None) for i in files]
C:\ProgramData\Miniconda3\envs\gnatss\lib\site-packages\gnatss\loaders.py:134: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  travel_times = [pd.read_csv(i, delim_whitespace=True, header=None) for i in files]
C:\ProgramData\Miniconda3\envs\gnatss\lib\site-packages\gnatss\loaders.py:134: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  travel_times = [pd.read_csv(i, delim_whitespace=True, header=None) for i in files]


,time,NCL1-1,NCL1-2,NCL1-3
0,712215465.0,2.263261,2.383398,2.400821
1,712215480.0,2.271801,2.379346,2.394874
2,712215495.0,2.281219,2.377755,2.388229
3,712215510.0,2.288577,2.371308,2.383921
4,712215525.0,2.301908,2.363096,2.382359
...,...,...,...,...
11995,712396326.0,2.258169,2.349168,2.437971
11996,712396341.0,2.247783,2.347241,2.450597
11997,712396356.0,2.243481,2.340243,2.463428
11998,712396371.0,2.244354,2.329481,2.473345


In [44]:
pxp_tt_merge_df = pxp_tt.merge(df, how="inner", left_on="time", right_on="Timestamp")
pxp_tt_merge_df

,time,NCL1-1,NCL1-2,NCL1-3,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status,Timestamp
0,712215465.0,2.263261,2.383398,2.400821,2220,322665.0,45.302841,-124.965797,-27.4228,0.4581,0.9704,0.3102,0.330828,1.106683,61.797842,INS_SOLUTION_GOOD,712215465.0
1,712215480.0,2.271801,2.379346,2.394874,2220,322680.0,45.302851,-124.965663,-26.7192,0.1621,0.6910,-0.4804,-1.300802,0.571030,64.206844,INS_SOLUTION_GOOD,712215480.0
2,712215495.0,2.281219,2.377755,2.388229,2220,322695.0,45.302887,-124.965537,-27.0523,0.0183,0.4897,0.0954,-0.821802,-0.433961,64.385785,INS_SOLUTION_GOOD,712215495.0
3,712215510.0,2.288577,2.371308,2.383921,2220,322710.0,45.302873,-124.965413,-27.3466,-0.3917,0.5685,-0.5390,-0.134781,1.034800,85.272367,INS_SOLUTION_GOOD,712215510.0
4,712215525.0,2.301908,2.363096,2.382359,2220,322725.0,45.302846,-124.965245,-25.9430,-1.1755,1.4913,0.4341,4.290541,-2.855544,96.191033,INS_SOLUTION_GOOD,712215525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,712396326.0,2.258169,2.349168,2.437971,2220,503526.0,45.302409,-124.965814,-22.6078,-0.5584,-0.4071,0.1908,-0.028292,1.839493,260.466165,INS_SOLUTION_GOOD,712396326.0
11996,712396341.0,2.247783,2.347241,2.450597,2220,503541.0,45.302321,-124.965968,-23.0234,-0.3902,-0.6366,-0.6910,3.047586,1.314111,220.728208,INS_SOLUTION_GOOD,712396341.0
11997,712396356.0,2.243481,2.340243,2.463428,2220,503556.0,45.302199,-124.966037,-23.0241,-0.6670,0.0372,-0.1427,-3.256572,5.628126,185.341521,INS_SOLUTION_GOOD,712396356.0
11998,712396371.0,2.244354,2.329481,2.473345,2220,503571.0,45.302068,-124.966026,-23.2353,-0.7742,-0.3284,0.3550,1.340106,-1.610324,138.660329,INS_SOLUTION_GOOD,712396371.0


In [ ]:
pxp